In [1]:
import re

切分文本：
spilt和正则表达式

In [24]:
def textParse(string):
    mySent = 'This is taoshao, From BeiJing University.'
    mySent.split() #标点被当成词的一部分
    regEx = re.compile('\\W*') # W* 0个或多个非字母数字或下划线字符（等价于[^a-zA-Z0-9_]）
    listOfTokens = regEx.split(mySent)
    # 去空白串 + 转为小写，方便词袋处理
    return [tok.lower() for tok in listOfTokens if len(tok) >0 ]